In [1]:
# Load AIEEE Data

In [1]:
!nvidia-smi

Sat Mar 19 01:53:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   39C    P0    43W / 250W |   5988MiB / 16280MiB |     42%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

In [3]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(0, 'Models/CBSEData/CBSEPreprocessing/')
from parse_cbse import *

In [5]:
cbse_df = CBSEData()

Load Models

In [11]:
#  other models
AIEEE_Muril = "Models/AIEEEGenderData/MurilBERT/SavedModel/MurilBERT"
CBSE_Muril = "Models/CBSEData/MurilBERT/SavedModel/MurilBERT"
ER_Muril = "Models/ERData/MurilBERT/SavedModel/BERTMuril"

In [7]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [8]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [9]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_Muril)

In [10]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_Muril)

In [12]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_Muril)

In [14]:
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")


Predict Gender

In [13]:
from transformers import pipeline
from sklearn.metrics import classification_report


In [18]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    print(classification_report(true, labels, digits = 4))
    return 

In [ ]:
print_results(cbse_model, tokenizer, cbse_df)

In [19]:
print_results(aieee_model, tokenizer, cbse_df)

              precision    recall  f1-score   support

         0.0     0.9677    0.9636    0.9657     67970
         1.0     0.9709    0.9742    0.9725     84519

    accuracy                         0.9695    152489
   macro avg     0.9693    0.9689    0.9691    152489
weighted avg     0.9695    0.9695    0.9695    152489



In [ ]:
print_results(er_model, tokenizer, cbse_df)